In [1]:
# ==================================================================
# A-Eye Model Forward Pass Test for a Jupyter Notebook
# ==================================================================
import torch
import cv2
import numpy as np
import os
import sys

# --- 1. CONFIGURATION ---
IMAGE_PATH = "C:/Users/denni/Downloads/test.png"

# --- 2. SETUP PATHS ---
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

try:
    # --- 3. IMPORT CUSTOM MODULES ---
    from radial_tokenizer.radial_tokenizer import RadialTokenizer
    from model.aeye_model import AEyeModel
    print("✅ Modules imported successfully.")
except ImportError as e:
    print(f"❌ ImportError: {e}")
    print("Please ensure your notebook is in the correct directory (e.g., 'A-Eye/model/')")
    print("and that the necessary '__init__.py' files exist in your folders.")

# --- 4. LOAD AND PREPROCESS IMAGE ---
print(f"\nLoading image from: {IMAGE_PATH}")
if not os.path.exists(IMAGE_PATH):
    raise FileNotFoundError(f"Error: Image not found at the specified path: {IMAGE_PATH}")

# Load image, resize to the model's expected input (128x128), and convert color
image = cv2.imread(IMAGE_PATH)
image_resized = cv2.resize(image, (128, 128))
image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)

# Normalize pixel values to [0.0, 1.0] and convert to a PyTorch tensor
image_norm = image_rgb / 255.0
# Rearrange dimensions from [H, W, C] to [B, C, H, W] for the model
image_tensor = torch.from_numpy(image_norm).permute(2, 0, 1).unsqueeze(0).float()

print(f"✅ Image loaded and preprocessed.")
print(f"Image tensor shape: {image_tensor.shape}")

# --- 5. GENERATE RADIAL TOKENS ---
print("\nGenerating radial tokens...")
# Initialize the tokenizer
tokenizer = RadialTokenizer()
# Generate the 9D tokens from the image tensor
tokens_9d = tokenizer(image_tensor)
print("✅ Radial tokens generated.")
print(f"Tokens shape: {tokens_9d.shape}")

# --- 6. INITIALIZE AND RUN THE A-EYE MODEL ---
print("\nInitializing the AEyeModel...")
model = AEyeModel()
model.eval()  # Set the model to evaluation mode
print("✅ Model initialized.")

print("\nRunning the model's forward pass...")
with torch.no_grad():  # Use no_grad for inference to save memory
    # Pass BOTH the image tensor and the 9D tokens to the model
    output = model(image_tensor, tokens=tokens_9d)
print("✅ Forward pass complete.")

# --- 7. DISPLAY RESULTS ---
print("\n" + "="*20 + " RESULTS " + "="*20)
print(f"Final output shape: {output.shape}")
# .squeeze() removes extra dimensions, .item() gets the scalar value
predicted_score = output.squeeze().item()
print(f"Predicted Raw Score: {predicted_score:.4f}")

# Interpret the result based on the sigmoid output
prediction_label = "Mature Cataract" if predicted_score > 0.5 else "Immature Cataract"
print(f"Prediction: {prediction_label} (threshold @ 0.5)")
print("\nIntegration test successful!")

✅ Modules imported successfully.

Loading image from: C:/Users/denni/Downloads/test.png
✅ Image loaded and preprocessed.
Image tensor shape: torch.Size([1, 3, 128, 128])

Generating radial tokens...
✅ Radial tokens generated.
Tokens shape: torch.Size([1, 4, 9])

Initializing the AEyeModel...
✅ Model initialized.

Running the model's forward pass...

Input image shape: torch.Size([1, 3, 128, 128])

Radial tokens shape: torch.Size([1, 4, 9])

=================STAGES==================

After stage1: torch.Size([1, 16, 64, 64])

After stage2: torch.Size([1, 32, 32, 32])

After stage3: torch.Size([1, 32, 32, 32])

After stage4: torch.Size([1, 64, 16, 16])

After stage5: torch.Size([1, 64, 16, 16])

After stage6: torch.Size([1, 96, 8, 8])

After stage7: torch.Size([1, 96, 8, 8])

After global pooling: torch.Size([1, 96])

Final output shape: torch.Size([1, 1])
✅ Forward pass complete.

==================== RESULTS ====================
Final output shape: torch.Size([1, 1])
Predicted Raw Scor

c:\Users\denni\Music\A-Eye\radial_tokenizer\radial_tokenizer.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  tokens_9d = torch.tensor(tokens_9d_list, dtype=torch.float32, device=device)
